In [1]:
#conda install scikit-learn

In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
pd.set_option('display.max_columns', 100)
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [28]:
df.shape

(250, 38)

In [29]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [30]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

In [31]:
df['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [32]:
# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

In [33]:
df['Genre']

0                 [crime,  drama]
1                 [crime,  drama]
2                 [crime,  drama]
3        [action,  crime,  drama]
4                 [crime,  drama]
                  ...            
245           [drama,  film-noir]
246                       [drama]
247    [comedy,  drama,  romance]
248           [biography,  drama]
249                       [drama]
Name: Genre, Length: 250, dtype: object

In [34]:
df['Director'] = df['Director'].map(lambda x: x.split(' '))

In [35]:
df['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [36]:
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [37]:
df['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [38]:
df['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [15]:
# pre-check for nltk

# pip install rake-nltk
# import nltk
# nltk.download()

In [16]:
from rake_nltk import Rake


In [17]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [18]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [19]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]","[Frank, Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",
The Godfather,"[crime, drama]","[Francis, Ford, Coppola]","[Marlon Brando, Al Pacino, James Caan]",
The Godfather: Part II,"[crime, drama]","[Francis, Ford, Coppola]","[Al Pacino, Robert Duvall, Diane Keaton]",
The Dark Knight,"[action, crime, drama]","[Christopher, Nolan]","[Christian Bale, Heath Ledger, Aaron Eckhart]",
12 Angry Men,"[crime, drama]","[Sidney, Lumet]","[Martin Balsam, John Fiedler, Lee J. Cobb]",


In [20]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

TypeError: can only concatenate str (not "list") to str

In [ ]:
df.head()

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

In [ ]:
count_matrix 

In [ ]:
c=count_matrix.todense()

In [ ]:
c

In [ ]:
type(count_matrix)

In [ ]:
print(count_matrix[0,:])

In [ ]:
count.vocabulary_

In [ ]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

In [ ]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Fargo')

In [ ]:
indices[indices == 'Fargo'].index[0]

In [ ]:
pd.Series(cosine_sim[indices[indices == 'Fargo'].index[0]])